### 决策树三种算法
- ID3：特征划分基于信息增益  
- C4.5：特征划分基于信息增益比
- CART：特征划分基于基尼指数

决策树的流程为：

　　(1)输入需要分类的数据集和类别标签和靶标签。

　　(2)检验数据集是否只有一列，或者是否最后一列(靶标签数据默认放到最后一列)只有一个水平(唯一值)。

　　　　是：返回唯一值水平或者占比最大的那个水平

　　(3)调用信息增益公式，计算所有节点的信息增益，得到最大信息增益所对应的类别标签。

　　(4)建立决策树字典用以保存当次叶节点数据信息。

　　(5)进入循环：

　　　　按照该类别标签的不同水平，依次计算子数据集；

　　　　对子数据集重复(1),(2),(3),(4),(5), (6)步。

　　(6)返回决策树字典。

决策树实际上是一个大的递归函数，其结果是一个多层次的字典。

### ID3

In [83]:
import pandas as pd
import numpy as np
import json

# class LoadDataSet(object):
#     def load_dataSet(self):
#         data = pd.read_csv('./ID3data.txt', sep='\t', header=None)
#         data.rename(columns={0: 'age', 1: 'income', 2: 'student', 3: 'reputation', 4: 'purchase'},inplace=True)
#         return data

# # 用来加载和存储训练的模型
# class TreeHandler(object):
#     def __init__(self):
#         self.tree = None
#     def save(self, tree):
#         self.tree = tree
#         with open('tree.txt', mode='w', encoding='utf-8') as f:
#             tree = json.dumps(tree, indent=" ", ensure_ascii=False) # ensure_ascii=False 可以输出中文
#             f.write(tree)
#     def load(self, file):
#         with open(file, mode='r', encoding='utf-8') as f:
#             tree = f.read()
#             self.tree = json.loads(tree)
#         return self.tree
    
# ID3
class ID3Tree(dataSet):
    def __init__(self):
        super().__init__()
        self.dataSet = self.load_dataSet()
        self.InfoGain = {}
    def _entropy(self, dataSet):
        # 计算给定数据集的熵
        labels = dataSet.columns.tolist()
        # 统计分类标签不同水平的值
        level_count = dataSet[labels[-1]].value_counts().to_dict()
        entropy = 0
        for level, nums in level_count.items():
            prob = float(nums) / dataSet.shape[0]
            entropy += -prob * np.log2(prob)
        return entropy
    def _split_dataSet(self, dataSet, best_label, level):
        # 根据给定的特征label和其分类level来获取新的子数据集
        subdata = dataSet[dataSet[best_label] == level]
        
        # 去掉best_label这一列，重置索引,drop=True去掉原索引
        del subdata[best_label]
        subdata.reset_index(drop=True, inplace=True)
        return subdata
        
    def _best_split(self, dataSet):
        # 计算每个特征的信息增益
        best_infoGain = 0.0
        best_label = None
        
        labels = dataSet.columns.tolist()[:-1]
        Entropy_S = self._entropy(dataSet) # 把标签的熵
        # label是新加的特征，level是label下的分类
        for index, label in enumerate(labels):
            levels = dataSet[label].unique().tolist()
            label_entropy = 0.0
            for level in levels:
                level_data = dataSet[dataSet[label] == level]
                prob = level_data.shape[0] / dataSet.shape[0]
                label_entropy += prob * self._entropy(level_data)
            infoGain = Entropy_S - label_entropy
            if infoGain > best_infoGain:
                best_infoGain = infoGain
                best_label = label
            self.InfoGain.setdefault(self.__count, {}) # 保存{__count: {label1: infoGain1, label2: infoGain2, ...}}
            self.InfoGain[self.__count][label] = infoGain
        self.__count += 1
        return best_label
    
    def _top_amount_level(self, target_list):
        level_count = target_list.unique().value_counts().to_dict()
        sorted_level_count = sorted(level_count.items(), key=lambda x: x[1], reverse=True)
        return sorted_level_count[0][0]
    
    def mktree(self, dataSet):
        target_list = dataSet.iloc[:, -1] # 靶标签的那一列数据
        # 程序终止条件一：靶标签在该数据集上只有一个值(全部归为1类)，返回该值
        if target_list.unique().shape[0] <= 1:
            return target_list[0]
        
        # 程序终止条件二：数据集只剩下靶标签这一列，返回靶标签上数量最多的水平,这是将所有的都归为这一类，少数服从多数
        if dataSet.shape[0] == 1:
            return self._top_amount_level(dataSet)
        
        # 递归
        # 选择最佳分类特征
        best_label = self._best_split(dataSet)
        
        # 在选择的特征下面选择分类
        best_label_levels = dataSet[best_label].unique().tolist()
        tree = {best_label: {}} # 生成字典，保存树分类信息
        for level in best_label_levels:
            level_subdata = self._split_dataSet(dataSet, best_label, level) #params: 原数据， 信息增益最大的特征， 特性下的分类level
            tree[best_label][level] = self.mktree(level_subdata)
        return tree
    
    def predict(self, tree, labels, test_samples):
        '''
        tree: trained model, a dict
        labels: all features
        '''
        firstStr = list(tree.keys())[0]
        secondDict = tree[firstStr]
        featIndex = labels.index(firstStr) # 找到第一个特征label在labels上的索引
        for key in secondDict.keys():
            if test_samples[featIndex] == key:
                if secondDict[key].__class__.__name__ == "dict":
                    classLabel = self.predict(secondDict[key], labels, test_samples)
                else:
                    classLabel = secondDict[key]
        return classLabel
        
    def _unit_test(self):
        """用于测试_entropy函数"""
        data = [[1, 1, "yes"], 
                [1, 1, "yes"],
                [1, 0, "no"],
                [0, 1, "no"],
                [0, 1, "no"],]
        data = pd.DataFrame(data=data, columns=["a", "b", "c"])
        self.tree = self.mktree(self.dataSet)
        labels = ["age", "income", "student", "reputation"]
        test_sample = [0, 1, 0, 1]   # [0, 1, 0, 0, "no"]
        output = self.predict(self.tree, labels, test_sample)
        print("The truth class is %s, The ID3Tree outcome is %s." % ("no", output))

In [84]:
model = ID3Tree()

In [85]:
model._unit_test()

The truth class is no, The ID3Tree outcome is no.


{
	"0": {
		"age": 0.2657121273840979,
		"income": 0.01774123883005596,
		"reputation": 0.04631324460790964,
		"student": 0.1738568696347308
	},
	"1": {
		"income": 0.4591479170272448,
		"reputation": 0.044110417748401076,
		"student": 0.9182958340544896
	},
	"2": {
		"income": 0.044110417748401076,
		"reputation": 0.9182958340544896,
		"student": 0.050484873918346995
	}
}
{
	"age": {
		"0": {
			"student": {
				"0": "no",
				"1": "yes"
			}
		},
		"1": "yes",
		"2": {
			"reputation": {
				"0": "yes",
				"1": "no"
			}
		}
	}
}


In [123]:
import pandas as pd
import numpy as np
import json
    
# ID3
class ID3Tree(object):
    def __init__(self):
        self.InfoGain = {}
        self.__count = 0
        self.tree = {}
        
    def _entropy(self, dataSet):
        # 计算给定数据集的熵
        labels = dataSet.columns.tolist()
        # 统计分类标签不同水平的值
        level_count = dataSet[labels[-1]].value_counts().to_dict()
        entropy = 0
        for level, nums in level_count.items():
            prob = float(nums) / dataSet.shape[0]
            entropy += -prob * np.log2(prob)
        return entropy
    
    def _split_dataSet(self, dataSet, best_label, level):
        # 根据给定的特征label和其分类level来获取新的子数据集
        subdata = dataSet[dataSet[best_label] == level]       
        # 去掉best_label这一列，重置索引,drop=True去掉原索引
        del subdata[best_label]
        subdata.reset_index(drop=True, inplace=True)
        return subdata
        
    def _best_split(self, dataSet):
        # 计算每个特征的信息增益
        best_infoGain = 0.0
        best_label = None     
        labels = dataSet.columns.tolist()[:-1]
        Entropy_S = self._entropy(dataSet) # 把标签的熵
        # label是新加的特征，level是label下的分类
        for index, label in enumerate(labels):
            levels = dataSet[label].unique().tolist()
            label_entropy = 0.0
            for level in levels:
                level_data = dataSet[dataSet[label] == level]
                prob = level_data.shape[0] / dataSet.shape[0]
                label_entropy += prob * self._entropy(level_data)
            infoGain = Entropy_S - label_entropy
            if infoGain > best_infoGain:
                best_infoGain = infoGain
                best_label = label
            self.InfoGain.setdefault(self.__count, {}) # 保存{__count: {label1: infoGain1, label2: infoGain2, ...}}
            self.InfoGain[self.__count][label] = infoGain
        self.__count += 1
        return best_label
    
    def _top_amount_level(self, target_list):
        level_count = target_list.unique().value_counts().to_dict()
        sorted_level_count = sorted(level_count.items(), key=lambda x: x[1], reverse=True)
        return sorted_level_count[0][0]
    
    def mktree(self, dataSet):
        target_list = dataSet.iloc[:, -1] # 靶标签的那一列数据
        # 程序终止条件一：靶标签在该数据集上只有一个值(全部归为1类)，返回该值
        if target_list.unique().shape[0] <= 1:
            return target_list[0]
        
        # 程序终止条件二：数据集只剩下靶标签这一列，返回靶标签上数量最多的水平,这是将所有的都归为这一类，少数服从多数
        if dataSet.shape[0] == 1:
            return self._top_amount_level(dataSet)
        
        # 递归
        # 选择最佳分类特征
        best_label = self._best_split(dataSet)
        
        # 在选择的特征下面选择分类
        best_label_levels = dataSet[best_label].unique().tolist()
        print("best_label")
        print(best_label)
        tree = {best_label: {}} # 生成字典，保存树分类信息
        for level in best_label_levels:
            level_subdata = self._split_dataSet(dataSet, best_label, level) #params: 原数据， 信息增益最大的特征， 特性下的分类level
            tree[best_label][level] = self.mktree(level_subdata)
        print("tree")
        print(tree)
        self.tree = tree
        
        
    def predict(self, labels, test_samples):
        '''
        tree: trained model, a dict
        labels: all features
        '''
        firstStr = list(self.tree.keys())[0]
        secondDict = self.tree[firstStr]
        featIndex = labels.index(firstStr) # 找到第一个特征label在labels上的索引
        for key in secondDict.keys():
            if test_samples[featIndex] == key:
                if secondDict[key].__class__.__name__ == "dict":
                    classLabel = self.predict(secondDict[key], labels, test_samples)
                else:
                    classLabel = secondDict[key]
        
        return classLabel
        
#     def _unit_test(self):
#         """用于测试_entropy函数"""
#         data = [[1, 1, "yes"], 
#                 [1, 1, "yes"],
#                 [1, 0, "no"],
#                 [0, 1, "no"],
#                 [0, 1, "no"],]
#         data = pd.DataFrame(data=data, columns=["a", "b", "c"])
#         self.tree = self.mktree(self.dataSet)
#         labels = ["age", "income", "student", "reputation"]
#         test_sample = [0, 1, 0, 1]   # [0, 1, 0, 0, "no"]
#         output = self.predict(self.tree, labels, test_sample)
#         print("The truth class is %s, The ID3Tree outcome is %s." % ("no", output))

In [124]:
# # 用来加载和存储训练的
# class TreeHandler(object):
#     def __init__(self):
#         self.tree = None
#     def save(self, tree):
#         self.tree = tree
#         with open('tree.txt', mode='w', encoding='utf-8') as f:
#             tree = json.dumps(tree, indent=" ", ensure_ascii=False) # ensure_ascii=False 可以输出中文
#             f.write(tree)
#     def load(self, file):
#         with open(file, mode='r', encoding='utf-8') as f:
#             tree = f.read()
#             self.tree = json.loads(tree)
#         return self.tree

In [126]:
data = pd.read_csv('./ID3data.txt', sep='\t', header=None)
data.rename(columns={0: 'age', 1: 'income', 2: 'student', 3: 'reputation', 4: 'purchase'},inplace=True)
data.head()

,age,income,student,reputation,purchase
0,0,0,0,0,no
1,0,0,0,0,no
2,0,0,0,0,no
3,0,0,0,0,no
4,0,0,0,0,no


In [127]:
model = ID3Tree()
model.mktree(data)
print(model.tree)
#model.predict()
labels = model.predict(["age", "income", "student", "reputation"], [0, 1, 0, 1])
#labels

best_label
age
best_label
student
tree
{'student': {0: 'no', 1: 'yes'}}
best_label
reputation
tree
{'reputation': {0: 'yes', 1: 'no'}}
tree
{'age': {0: None, 1: 'yes', 2: None}}
{'age': {0: None, 1: 'yes', 2: None}}
